In [3]:
import sqlite3
conn = sqlite3.connect("AQI.db")
with conn:
    sql = '''
    CREATE TABLE IF NOT EXISTS records (
        id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
        sitename TEXT NOT NULL,
        county TEXT,
        aqi INTEGER,
        status TEXT,
        pm25 NUMERIC,
        date TEXT,
        lat NUMERIC,
        lon NUMERIC,
        UNIQUE(sitename,date)
    );
    '''
    cursor = conn.cursor()
    cursor.execute(sql)

In [4]:
import requests

url = 'https://data.moenv.gov.tw/api/v2/aqx_p_488?api_key=e8dd42e6-9b8b-43f8-991e-b3dee723a52d&limit=1000&sort=datacreationdate%20desc&format=JSON'
try:
    response = requests.get(url)
    response.raise_for_status()
    data = response.json()
except Exception as e:
    print(e)
else:
    sitenames = set()
    for items in data['records']:
        sitenames.add(items['sitename'])

    sitenames = list(sitenames)
print(sitenames)

['中壢', '彰化（員林）', '士林', '冬山', '臺東', '宜蘭', '南投', '大里', '崙背', '新竹', '復興', '線西', '花蓮', '潮州', '頭份', '中山', '基隆', '菜寮', '古亭', '彰化', '臺西', '新港', '永和', '松山', '淡水', '大園', '小港', '新莊', '林園', '橋頭', '新店', '土城', '二林', '宜蘭（頭城）', '苗栗', '前鎮', '萬里', '三重', '竹東', '屏東(枋山)', '左營', '新營', '西屯', '觀音', '鳳山', '富貴角', '板橋', '美濃', '恆春', '關山', '屏東', '仁武', '萬華', '埔里', '嘉義', '前金', '高雄（湖內）', '桃園', '湖口', '朴子', '麥寮', '安南', '汐止', '陽明', '忠明', '楠梓', '豐原', '新北(樹林)', '屏東（琉球）', '大寮', '馬公', '龍潭', '大城', '馬祖', '斗六', '林口', '沙鹿', '大同', '竹山', '三義', '臺南（麻豆）', '臺南', '平鎮', '善化', '金門']


In [5]:
import datasource
datasource.get_selected_data(sitename='屏東(枋山)')

[['2024-11-04 10:00', '屏東縣', '34', '18', '良好', '22.260899', '120.651472'],
 ['2024-11-04 10:00', '屏東縣', '34', '18', '良好', '22.260899', '120.651472'],
 ['2024-11-04 09:00', '屏東縣', '32', '5', '良好', '22.260899', '120.651472'],
 ['2024-11-04 09:00', '屏東縣', '32', '5', '良好', '22.260899', '120.651472'],
 ['2024-11-04 08:00', '屏東縣', '33', '6', '良好', '22.260899', '120.651472'],
 ['2024-11-04 08:00', '屏東縣', '33', '6', '良好', '22.260899', '120.651472'],
 ['2024-11-04 07:00', '屏東縣', '33', '5', '良好', '22.260899', '120.651472'],
 ['2024-11-04 07:00', '屏東縣', '33', '5', '良好', '22.260899', '120.651472'],
 ['2024-11-04 06:00', '屏東縣', '35', '4', '良好', '22.260899', '120.651472'],
 ['2024-11-04 06:00', '屏東縣', '35', '4', '良好', '22.260899', '120.651472'],
 ['2024-11-04 05:00', '屏東縣', '35', '3', '良好', '22.260899', '120.651472'],
 ['2024-11-04 05:00', '屏東縣', '35', '3', '良好', '22.260899', '120.651472']]

In [6]:
import sqlite3
conn = sqlite3.connect("AQI.db")
with conn:
    sql = '''INSERT OR IGNORE INTO records(sitename,county,aqi,status,pm25,date,lat,lon)
    values ('屏東(枋山)','屏東縣', 34, '良好',18,'2024-11-04 10:00', 22.260899, 120.651472);
    '''
    cursor = conn.cursor()
    cursor.execute(sql)

In [7]:
import sqlite3
conn = sqlite3.connect("AQI.db")
url = 'https://data.moenv.gov.tw/api/v2/aqx_p_488?api_key=e8dd42e6-9b8b-43f8-991e-b3dee723a52d&limit=1000&sort=datacreationdate%20desc&format=JSON'
try:
    response = requests.get(url)
    response.raise_for_status()
    data = response.json()
except Exception as e:
    print(e)
else:
    sitenames = set()
    with conn:
        cursor = conn.cursor()
        for items in data['records']:
            sitename = items['sitename']
            county = items['county']
            aqi = int(items['aqi']) if items['aqi'] != '' else 0
            status = items['status']
            pm25 = float(items['pm2.5']) if items['pm2.5'] != '' else 0.0
            date = items['datacreationdate']
            lon = float(items['longitude']
                        ) if items['longitude'] != '' else 0.0
            lat = float(items['latitude']) if items['latitude'] != '' else 0.0
            sql = '''INSERT OR IGNORE INTO records(sitename,county,aqi,status,pm25,date,lat,lon)
                    values (?,?, ?, ?,?,?,?,?);
            '''
            cursor.execute(sql, (sitename, county, aqi,
                           status, pm25, date, lat, lon))